# EA979 Final Project

## Pedestrian Recognition: A HOG algorithm improved by frame dropping algorithm

### Authors:
André Barros de Medeiros

Luca

Francesco

Inspired by Andrew Rosebrock @ https://www.pyimagesearch.com/2015/11/09/pedestrian-detection-opencv/

  OpenCV ships with a pre-trained HOG + Linear SVM model based, on Dalal and Triggs method to automatically detect pedestrians in images, that can be used to perform pedestrian detection in both images and video streams. Below we give an outline of the two:

## Dependencies: 
    - OpenCV
    - Numpy
    - argparse
    - imutils (for imutils in Anaconda: conda install -c conda-forge imutils)
    - scipy.stats
    - time
  
## Histogram of Oriented Gradients (HOG)

While OpenCV's Cascade Classifiers are fast, they leave much to desire. That's where HOG comes in. But if you want to check out cascade classifiers, a java application for face recognition can be found [here](https://github.com/andre91998/JavaBasics/tree/master/FaceDetection).

### HOG algorithm:

<ul><li><strong>Step 1:</strong> Sample P positive samples from your training data of the object(s) you want to detect and extract HOG descriptors from these samples. </ul></li>

<ul><li><strong>Step 2:</strong> Sample N negative samples from a negative training set that does not contain any of the objects you want to detect and extract HOG descriptors from these samples as well. In practice N >> P.</ul></li>

<ul><li><strong>Step 3:</strong> Train a <strong>Linear Support Vector Machine</strong> on your positive and negative samples.</ul></li>

<ul><li><strong>Step 4:</strong> Apply <em>hard-negative mining</em>. For each image and each possible scale of each image in your negative training set, apply the sliding window technique and slide your window across the image. At each window compute your HOG descriptors and apply your classifier. If your classifier (incorrectly) classifies a given window as an object (and it will, there will absolutely be false-positives), record the feature vector associated with the false-positive patch along with the probability of the classification. </ul></li>

<ul><li><strong>Step 5:</strong> Take the false-positive samples found during the hard-negative mining stage, sort them by their confidence (i.e. probability) and re-train your classifier using these hard-negative samples. (Note: You can iteratively apply steps 4-5, but in practice one stage of hard-negative mining usually [not not always] tends to be enough. The gains in accuracy on subsequent runs of hard-negative mining tend to be minimal.)</ul></li>

<ul><li><strong>Step 6: </strong>Your classifier is now trained and can be applied to your test dataset.</ul></li>

## Linear Support Vector Machines (SVM)

SVMs are supervised learning models with associated learning algorithms that analyze data used for classification and regression analysis. More formally, a support-vector machine constructs a hyperplane or set of hyperplanes in a high- or infinite-dimensional space, which can be used for classification, regression, or other tasks like outliers detection.

In the **linear** case (ours), the goal is to find the *"maximum-margin hyperplane"* that divides the data points into their correct classes, which means we want the distance between the closest data point of each class to be maximized.

## Our Idea:

In this project, we use Pearson's Correlation Coeficient to evaluate how similar each frame is to the one before. This allowed us to establish criteria for dropping frames similiar to the previous, drastically lessening the amount of processing power needed (which in turn increased the speed of the algorithm).

With PCC, each frame was attributed a value (coeficient). To decide wether or not to discard each frame, we compared the calculated value with a threshold value. If the coeficient was less than the threshold, than we re-applied the recognition algorithm, if not, then we just repeated the previously calculated identification rectangles (drawn around the identified people) on the new frame.

On top of that, instead of setting a fixed threshold for every image, we decided to implement a threshold that would change through time. By establishing this dynamic threshold (through the updated mean of the PC coeficients), we further reduced the amount of frames to be processed, by dropping them when they are similar enough to the one before, maintaining a non-deterministic approach (and increasing accuracy and speed).

In [ ]:
# -*- coding: utf-8 -*-
"""
@author: Andre Barros de Medeiros
@Date:29/30/2020
@Copyright: Free to use, copy and modify
"""

# import the necessary packages
from __future__ import print_function
from collections import deque
from imutils.object_detection import non_max_suppression
from imutils.video import VideoStream
from imutils import paths
from scipy.stats import pearsonr
import numpy as np
import argparse
import imutils
import cv2
import time


#initialize frame counter
counter = 0
pts = deque(maxlen=32)

coef = (0,0)

vs = cv2.VideoCapture("/Users/Andre/Documents/GitHub/pedestrian_recognition/method4/pedestrian.mp4")

# allow the camera or video file to warm up
time.sleep(2.0)

# initialize the HOG descriptor/person detector
hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

old_frame = None
# keep looping
while True:

    # grab the current frame
    frame = vs.read()
    
    # handle the frame from VideoCapture or VideoStream
    frame = frame[1]
    
    # if we are viewing a video and we did not grab a frame,
    # then we have reached the end of the video
    if frame is None:
        break
    
    if old_frame is None:
        
        # resize image it to (1) reduce detection time and (2) improve detection accuracy
        frame = imutils.resize(frame, width=min(400, frame.shape[1]))
        orig = frame.copy()
        old_frame = orig.flatten() #update old frame

        # detect people in the image
        (rects, weights) = hog.detectMultiScale(frame, winStride=(7,7), 
                                                padding=(4,4), scale=1.3)
        
        # draw the original bounding boxes
        for (x, y, w, h) in rects: 
            cv2.rectangle(orig, (x, y), (x + w, y + h), (0, 0, 255), 2)
        
        # apply non-maxima suppression to the bounding boxes using a
        # fairly large overlap threshold to try to maintain overlapping
        # boxes that are still people
        rects = np.array([[x, y, x + w, y + h] for (x, y, w, h) in rects])
        pick = non_max_suppression(rects, probs=None, overlapThresh=0.65)
        
        # draw the final bounding boxes
        for (xA, yA, xB, yB) in pick:
        		cv2.rectangle(frame, (xA, yA), (xB, yB), (0, 255, 0), 2)
        
        # show the frame
        #cv2.imshow("Before NMS", orig)
        cv2.imshow("After NMS", frame)
        
        # increment counter
        counter += 1
        
        # if the 'q' key is pressed, stop the loop
        key = cv2.waitKey(1) & 0xFF 
        # (& 0xFF) keeps last 8 bits of  waitKey output
        if key == ord("q"): break

    if old_frame is not None:
        
        # resize image it to (1) reduce detection time and (2) improve detection accuracy
        frame = imutils.resize(frame, width=min(400, frame.shape[1]))
        
        #flatten current frame for running the Pearson's Correlation 
        flat_frame = frame.flatten()
        #calculate the pearson's correlation coeficient
        coef = pearsonr(flat_frame, old_frame)
        
        #if on second frame, create threshold_arr for holding the PCCs
        if (counter == 1): 
            threshold_arr = np.array(coef[0])
            
        #if on any other frame, append to the array
        else:
            threshold_arr = np.append(threshold_arr, coef[0])
            
        #dynamical threshold calcuation: mean of all previous PCCs    
        threshold = np.mean(threshold_arr)
        print(coef,threshold)
        
        #if PCC below the threshold, re-classify
        if (((coef[0] < threshold)and(coef[0]>0))or((coef[0]>-1*threshold)and(coef[0]<0))):
            orig = frame.copy()
            old_frame = orig.flatten() #update old_frame
            
            # detect people in the image
            (rects, weights) = hog.detectMultiScale(frame, winStride=(1,1), 
                                                    padding=(8,8), scale=1.5)
            
            # draw the original bounding boxes
            for (x, y, w, h) in rects: 
                cv2.rectangle(orig, (x, y), (x + w, y + h), (0, 0, 255), 2)
            
            # apply non-maxima suppression to the bounding boxes using a
            # fairly large overlap threshold to try to maintain overlapping
            # boxes that are still people
            rects = np.array([[x, y, x + w, y + h] for (x, y, w, h) in rects])
            pick = non_max_suppression(rects, probs=None, overlapThresh=0.65)
            
            # draw the final bounding boxes
            for (xA, yA, xB, yB) in pick:
            		cv2.rectangle(frame, (xA, yA), (xB, yB), (0, 255, 0), 2)
            
            # show the frame
            #cv2.imshow("Before NMS", orig)
            cv2.imshow("After NMS", frame)
            
            # increment counter and update last frame
            counter += 1
            
            # if the 'q' key is pressed, stop the loop
            key = cv2.waitKey(1) & 0xFF 
            # (& 0xFF) keeps last 8 bits of  waitKey output
            if key == ord("q"): break
        
        # if PCC is above threshold, update frame, but keep same rectangles
        else:
            # draw the same rectangle as before on the current frame (which wasn't proccessed by HoG)
            for (xA, yA, xB, yB) in pick:
                cv2.rectangle(frame, (xA, yA), (xB, yB), (0, 255, 0), 2)
            
            cv2.imshow("After NMS", frame)
            counter += 1
            
            # if the 'q' key is pressed, stop the loop
            key = cv2.waitKey(1) & 0xFF 
            # (& 0xFF) keeps last 8 bits of  waitKey output
            if key == ord("q"): break
            
        
# if we are not using a video file, stop the camera video stream
if not args.get("video", False): vs.stop()

# otherwise, release the camera
else: vs.release()

# close all windows
cv2.destroyAllWindows()

(0.9999999999999963, 0.0) 0.9999999999999963
(0.9999908519433329, 0.0) 0.9999954259716646
(0.9999649767325043, 0.0) 0.9999852762252779
(0.999946229554237, 0.0) 0.9999755145575177
(0.9999389095460005, 0.0) 0.9999681935552143
(0.9999472285694058, 0.0) 0.9999646993909129
(0.9999077154721001, 0.0) 0.9999565588310825
(0.9999043878076271, 0.0) 0.9999500374531505
(0.9998605896414907, 0.0) 0.9999400988074105
(0.999928229275057, 0.0) 0.9999389118541752
(0.9999404916163674, 0.0) 0.9999390554689199
(0.9998962346052922, 0.0) 0.9999354870636177
(0.9999429134036737, 0.0) 0.999936058320545
(0.9998884001267396, 0.0) 0.9999326541638446
(0.9999547526641501, 0.0) 0.9999341273971983
(0.9996967049875547, 0.0) 0.9999192884965957
(0.9998815057422822, 0.0) 0.9999170659816362
(0.9999326856335535, 0.0) 0.999917933740076
(0.9998498229584998, 0.0) 0.9999143489620983
(0.9999472314471822, 0.0) 0.9999159930863526
(0.9998942298662151, 0.0) 0.9999149567425365
(0.9999532857647208, 0.0) 0.9999166989708176
(0.99991000713